# Journalists killed worldwide since 1992

Journalism is gathering information and presenting it in front of people in the form of news and writings. Journalists perform these tasks and even scarifice their lives for thier job. This dataset used for analysis was collected by CPJ since 1992 till 2016 and contains 18 variables giving the details about journalists, their work and their death.

In [1]:
# load necessary packages
library(janitor)
library(dplyr)
library(lubridate)
library(tidyverse)
library(ggplot2)


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ stringr 1.4.0
✔ tidyr   1.1.3     ✔ forcats 0.5.0
✔ readr   1.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::set

In [2]:
# loading the data
j_data <- read.csv("../input/journalists-killed-worldwide-since-1992/cpj.csv")

In [3]:
# checking first few rows of dataset
head(j_data)

,Type,Date,Name,Sex,Country_killed,Organization,Nationality,Medium,Job,Coverage,Freelance,Local_Foreign,Source_fire,Type_death,Impunity_for_murder,Taken_captive,Threatened,Tortured
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,Motive Confirmed,"October 22, 2016",Ali Risan,Male,Iraq,Al-Sumaria TV,Iraqi,"Television, Internet",Camera Operator,War,No,Local,Political Group,Crossfire/Combat-Related,,,,
2,Motive Confirmed,"October 21, 2016",Ahmet Haceroğlu,Male,Iraq,Türkmeneli TV,Iraqi,Television,Producer,"Politics, War",No,Local,Political Group,Crossfire/Combat-Related,,,,
3,Motive Confirmed,"October 2, 2016",Jeroen Oerlemans,Male,Libya,Freelance,Netherlands,"Print, Internet",Photographer,"Politics, War",Yes,Foreign,Political Group,Crossfire/Combat-Related,,,,
4,Motive Confirmed,"August 14, 2016",Mustafa Said,Male,Iraq,Kurdistan TV,Iraqi,Television,Camera Operator,War,No,Local,Unknown Fire,Crossfire/Combat-Related,,No,No,No
5,Motive Confirmed,"August 8, 2016",Shehzad Ahmed,,Pakistan,Aaj News,Pakistani,Television,Camera Operator,"Business, Corruption, Crime, Culture, Human Rights, Politics, Sports",No,Local,Political Group,Dangerous Assignment,,No,No,No
6,Motive Confirmed,"August 8, 2016",Mehmood Khan,Male,Pakistan,DawnNews,Pakistani,"Television, Internet",Camera Operator,"Business, Corruption, Crime, Culture, Human Rights, Politics, Sports",No,Local,Political Group,Dangerous Assignment,,No,No,No


In [4]:
summary(j_data) # checking summary of data
data <- j_data 

                 Type                     Date     
 Media Workers     :  96   November 23, 2009:  32  
 Motive Confirmed  :1216   October 12, 2006 :  12  
 Motive Unconfirmed: 470   January  7, 2015 :   8  
                           December 23, 2013:   5  
                           January 25, 2015 :   5  
                           June  1, 1994    :   5  
                           (Other)          :1715  
                             Name          Sex           Country_killed
 Maher                         :   2         : 566   Iraq       : 264  
 Unidentified                  :   2   Female:  80   Philippines: 133  
  Ahmad [full name unavailable]:   1   Male  :1136   Syria      : 116  
  Khaled Mohammad Nofan        :   1                 Mexico     :  89  
  Sabah Salman                 :   1                 Pakistan   :  86  
  Unidentified                 :   1                 Colombia   :  84  
 (Other)                       :1774                 (Other)    :1010  
     Org

In [5]:
# removing empty levels
data$Taken_captive <- factor(data$Taken_captive,exclude='')
data$Threatened <- factor(data$Threatened, exclude='')
data$Tortured <- factor(data$Tortured, exclude='\n')
data$Type_death <- factor(data$Type_death, exclude='')
data$Sex <- factor(data$Sex,exclude='')
data$Freelance <- factor(data$Freelance,exclude='')
data$Local_Foreign <- factor(data$Local_Foreign,exclude='')
data$Impunity_for_murder <- factor(data$Impunity_for_murder, exclude='')
data$Coverage <- factor(data$Coverage, exclude='')
data$Job <- factor(data$Job, exclude='')
data$Medium <- factor(data$Medium, exclude='')

In [6]:
# correcting the data
data <- data %>% 
    mutate(Taken_captive=ifelse(as.character(Taken_captive)=='Yes ','Yes', as.character(Taken_captive))) %>%
    mutate(Threatened=ifelse(as.character(Threatened)==' Yes','Yes',ifelse(as.character(Threatened)=='No ','No',as.character(Threatened)))) %>%
    mutate(Tortured=ifelse(as.character(Tortured)=='No\n','No',ifelse(as.character(Tortured)=='Yes\n','Yes',as.character(Tortured)))) %>%
    mutate(Taken_captive=as.factor(Taken_captive), Threatened = as.factor(Threatened), Tortured=as.factor(Tortured)) 

In [7]:
# creating seperate columns for year and month
data <- data %>% 
    mutate(Date=as.Date(Date, format="%B%d,%Y")) %>%
    mutate(Month=month(ymd(Date)), Year=year(ymd(Date)))

In [8]:
# removing the outlier
data <- data %>%
    filter(!Year==2030)

In [9]:
# gathering the data to make dataset long
data <- data %>% 
     mutate(Impunity_for_murder=as.character(Impunity_for_murder), Taken_captive=as.character(Taken_captive), Threatened
          =as.character(Threatened), Tortured = as.character(Tortured)) %>%
    gather(key='Source_of_death', value='Yes_No',Impunity_for_murder:Tortured) %>%
    mutate(Source_of_death=as.factor(Source_of_death), Yes_No=as.factor(Yes_No))

In [10]:
summary(data)

                 Type           Date           
 Media Workers     : 380   Min.   :3-05-02     
 Motive Confirmed  :4684   1st Qu.:2000-05-02  
 Motive Unconfirmed:1776   Median :2006-12-31  
                           Mean   :2003-07-19  
                           3rd Qu.:2011-12-01  
                           Max.   :2016-10-22  
                                               
                             Name          Sex           Country_killed
 Maher                         :   8   Female: 308   Iraq       :1040  
 Unidentified                  :   8   Male  :4376   Philippines: 532  
  Ahmad [full name unavailable]:   4   NA's  :2156   Syria      : 420  
  Khaled Mohammad Nofan        :   4                 Colombia   : 336  
  Sabah Salman                 :   4                 Pakistan   : 324  
  Unidentified                 :   4                 Russia     : 312  
 (Other)                       :6808                 (Other)    :3876  
     Organization       Nationality     

[Link to tableau visualization](https://public.tableau.com/profile/shilpi.dubey#!/vizhome/Journalistdeathanalysis/Deathanalysis)

Exploratory data analysis on journalists-killed-worldwide-since 1992 dataset resulted in several points to be noticed related to journalists deaths,
* Throughout the world, Iraq has witnessed most number of journalists deaths but the trend there started since 2003.
* Most common death practice on journalists has always been murder but with time it has shifted towards crossfire/combat related death practices.
* Male journalists are more likely to be killed may be foreign or local but most of the deaths reported are local.
* There has always been motive behind journalists killing and those killed worked for Freelancer organization mainly. This shows that journalists working for Freelancer organization have less security. 
* Coverage area for journalists have been war and politics mostly, leading to killing of journalists working there.
* Journalists generally work for television and print media, covering war and politics respectively and therefore, these fields have witnessed more deaths than any other medium of publishing. 
* For their work, they have been given impunity for murder followed by threatning and captivating.

Journalists have been working very hard to provide us the right news and the coverage, so it's there right to be protected from unwanted killing and deaths.